In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import numpy as np
import cc3d
import cv2
from PIL import Image
import pandas as pd
from tqdm.notebook import tqdm
from skimage import morphology
from skimage.measure import regionprops, label, find_contours
from scipy.spatial.distance import cdist
import scipy.ndimage as ndimage

In [2]:
def normalize_x(image):
    return image / 127.5 - 1


def denormalize_x(image):
    return (image + 1) * 127.5


def normalize_y(image):
    return image / 255


def denormalize_y(image):
    return image * 255

In [3]:
def load_X_gray(folder_path):
    
    image_files = []       
    for file in os.listdir(folder_path):
        base, ext = os.path.splitext(file)
        if ext == '.png':
            image_files.append(file)
        else :
            pass
        
    image_files.sort()
    
    img = cv2.imread(folder_path + os.sep + image_files[0], cv2.IMREAD_GRAYSCALE)
    
    images = np.zeros((len(image_files), img.shape[0], img.shape[1], 1), np.float32)
    for i, image_file in enumerate(image_files):
        image = cv2.imread(folder_path + os.sep + image_file, cv2.IMREAD_GRAYSCALE)
        image = image[:, :, np.newaxis]
        images[i] = normalize_x(image)
    
    print(images.shape)
    
    return images, image_files


def load_Y_gray(folder_path, thresh = None , normalize = False):
    
    image_files = []
    for file in os.listdir(folder_path):
        base, ext = os.path.splitext(file)
        if ext == '.png':
            image_files.append(file)
        else:
            pass
        
    image_files.sort()
    
    img = cv2.imread(folder_path + os.sep + image_files[0], cv2.IMREAD_GRAYSCALE)
    
    images = np.zeros((len(image_files), img.shape[0], img.shape[1], 1) ,np.float32)
    
    for i , image_file in enumerate(image_files):
        image = cv2.imread(folder_path + os.sep + image_file, cv2.IMREAD_GRAYSCALE)
        if thresh:
            ret , image = cv2.threshold(image , thresh , 255 , cv2.THRESH_BINARY)
        image = image[:, :, np.newaxis]
        if normalize:
            images[i] = normalize_y(image)
        else:
            images[i] = image
            
    print(images.shape)
    
    return images, image_files

In [4]:
def pick_counters(imgs):
    
    """
        Args : 
            imgs (numpy.ndarray) : Z, Y, X, 1
        Returns : 
            counter_imgs (numpy.ndarray) : Z, Y, X, 1
    """
    
    counter_imgs = np.zeros_like(imgs, np.float32)
    for z in range(imgs.shape[0]):
        contours, hierarchy = cv2.findContours(imgs[z].astype("uint8"), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        img = np.zeros((imgs.shape[1], imgs.shape[2], 1), np.float32)
        line = cv2.drawContours(img, contours, -1, (1,0,0), 1)
        
        line[0, :] = 0
        line[-1, :] = 0
        line[:, 0] = 0
        line[:, -1] = 0
        
        counter_imgs[z] = line
    return counter_imgs


def pick_counters_3d(imgs):
    
    """
        Args : 
            imgs (numpy.ndarray) : Z, Y, X, 1
        Returns : 
            counter_imgs (numpy.ndarray) : Z, Y, X, 1
        Note : 
            isotropicじゃないと使えません。端っこの面は塗りつぶさないように工夫しています。
    """
    
    z_counter_imgs = pick_counters(imgs)
    y_counter_imgs = pick_counters(imgs.transpose(1, 2, 0, 3))
    x_counter_imgs = pick_counters(imgs.transpose(2, 0, 1, 3))
    
    merged_counter_img = z_counter_imgs + y_counter_imgs.transpose(2, 0, 1, 3) + x_counter_imgs.transpose(1, 2, 0, 3)
    
    merged_counter_img[0, : ,:] = z_counter_imgs[0]
    merged_counter_img[-1, : ,:] = z_counter_imgs[-1]
    merged_counter_img[:, 0, :] = y_counter_imgs.transpose(2, 0, 1, 3)[:, 0, :]
    merged_counter_img[:, -1, :] = y_counter_imgs.transpose(2, 0, 1, 3)[:, -1, :]
    merged_counter_img[:, :, 0] = x_counter_imgs.transpose(1, 2, 0, 3)[:, :, 0]
    merged_counter_img[:, :, -1] = x_counter_imgs.transpose(1, 2, 0, 3)[:, :, -1]
    
    return (merged_counter_img>0).astype(np.float32)


def dilate_imgs(imgs):
    
    
    """
        Args : 
            imgs (numpy.ndarray) : Z, Y, X, 1
            
        Returns : 
            dilated_imgs (numpy.ndarray) : Z, Y, X, 1
    """
    
    dilated_imgs = morphology.binary_dilation(imgs[:,:,:,0], morphology.ball(3)).astype(np.uint8)

    return dilated_imgs


def quantify_mercs(labeled_mito_imgs, mercs_imgs):
    
    
    """
        Args : 
            labeled_mito_imgs (numpy.ndarray) : Z, Y, X, 1
            mercs_imgs (numpy.ndarray) : Z, Y, X, 1
            
        Returns : 
            mercs_volume (num)
    """    
    
    mercs_volume = np.count_nonzero((labeled_mito_imgs > 0) & (mercs_imgs > 0))
    
    return mercs_volume

In [5]:
def main(mito_path, out_dir, out_filename):
    mito_imgs, _ = load_Y_gray(mito_path)
    #er_imgs, _ = load_Y_gray(mercs_path)
    
    ## mercs部分の抽出準備
    #er_imgs = np.where(mito_imgs>0, 0, er_imgs)
    #dilated_er_imgs = dilate_imgs(er_imgs)
    
    ## mitoをラベル化
    connectivity = 6 # 縦横下
    labels_mito_imgs = cc3d.connected_components((mito_imgs[:,:,:,0] == 1).astype(int), connectivity = connectivity)
    
    ## 定量する
    df = pd.DataFrame(
        index=[], 
        columns=['mito_No', 'mito_V_px', "mito_V_nm", 'mito_S_px', "mito_S_nm"],
        dtype=object
    )
    for i in tqdm(range(1, int(np.max(labels_mito_imgs) + 1))):
        target_mito = (labels_mito_imgs==i).astype("uint8")[..., np.newaxis]
        target_mito_counters = pick_counters(target_mito)
        #target_mercs = np.where(target_mito_counters[:,:,:,0]>0, dilated_er_imgs, 0)
        
        mito_V = np.sum(target_mito)
        mito_S = np.sum(target_mito_counters)
        #mercs_S = np.sum(target_mercs)
        #mercs_ratio = mercs_S/mito_S
        
        df = df.append(pd.Series({
            'mito_No'           : i, 
            'mito_V_px'        : int(mito_V), 
            "mito_V_nm"      : int(mito_V)*12.5*12.5*50, 
            'mito_S_px'        : int(mito_S), 
            "mito_S_nm"      : int(mito_S)* 50* 12.5
        }), ignore_index = True)
    
    os.makedirs(out_dir, exist_ok = True)
    df.to_csv(f"{out_dir}/{out_filename}")
    

In [6]:
gene = "02_PDZD8KO"
crop_no = "Cropped_01"

BASE =  f"/home/neurobiology/drobo/DeepLearningData/083_KN_Exp313-AT/annotations/{gene}/{crop_no}/"
mito_path = f"{BASE}/selected_mito/"
#mercs_path = f"{BASE}/modified_mercs"
out_dir = f"/home/neurobiology/drobo/DeepLearningData/083_KN_Exp313-AT/Data/{gene}/{crop_no}/"
out_filename = "mito_001_non3D.csv"


if __name__ == "__main__":
    main(mito_path, out_dir, out_filename)

(74, 2376, 2424, 1)


  0%|          | 0/36 [00:00<?, ?it/s]

/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Seri

In [7]:
gene = "02_PDZD8KO"
crop_no = "Cropped_02"

BASE =  f"/home/neurobiology/drobo/DeepLearningData/083_KN_Exp313-AT/annotations/{gene}/{crop_no}/"
mito_path = f"{BASE}/selected_mito/"
#mercs_path = f"{BASE}/modified_mercs"
out_dir = f"/home/neurobiology/drobo/DeepLearningData/083_KN_Exp313-AT/Data/{gene}/{crop_no}/"
out_filename = "mito_001_non3D.csv"

if __name__ == "__main__":
    main(mito_path, out_dir, out_filename)

(74, 1056, 1320, 1)


  0%|          | 0/27 [00:00<?, ?it/s]

/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Seri

In [8]:
gene = "02_PDZD8KO"
crop_no = "Cropped_03"

BASE =  f"/home/neurobiology/drobo/DeepLearningData/083_KN_Exp313-AT/annotations/{gene}/{crop_no}/"
mito_path = f"{BASE}/selected_mito/"
#mercs_path = f"{BASE}/modified_mercs"
out_dir = f"/home/neurobiology/drobo/DeepLearningData/083_KN_Exp313-AT/Data/{gene}/{crop_no}/"
out_filename = "mito_001_non3D.csv"


if __name__ == "__main__":
    main(mito_path, out_dir, out_filename)

(74, 916, 1600, 1)


  0%|          | 0/8 [00:00<?, ?it/s]

/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Seri

In [9]:
gene = "02_PDZD8KO"
crop_no = "Cropped_04"

BASE =  f"/home/neurobiology/drobo/DeepLearningData/083_KN_Exp313-AT/annotations/{gene}/{crop_no}/"
mito_path = f"{BASE}/selected_mito/"
#mercs_path = f"{BASE}/modified_mercs"
out_dir = f"/home/neurobiology/drobo/DeepLearningData/083_KN_Exp313-AT/Data/{gene}/{crop_no}/"
out_filename = "mito_001_non3D.csv"


if __name__ == "__main__":
    main(mito_path, out_dir, out_filename)

(74, 1428, 2060, 1)


  0%|          | 0/15 [00:00<?, ?it/s]

/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Seri

In [10]:
gene = "02_PDZD8KO"
crop_no = "Cropped_05"

BASE =  f"/home/neurobiology/drobo/DeepLearningData/083_KN_Exp313-AT/annotations/{gene}/{crop_no}/"
mito_path = f"{BASE}/selected_mito/"
#mercs_path = f"{BASE}/modified_mercs"
out_dir = f"/home/neurobiology/drobo/DeepLearningData/083_KN_Exp313-AT/Data/{gene}/{crop_no}/"
out_filename = "mito_001_non3D.csv"


if __name__ == "__main__":
    main(mito_path, out_dir, out_filename)

(74, 2172, 3078, 1)


  0%|          | 0/13 [00:00<?, ?it/s]

/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Seri

In [11]:
gene = "01_Control"
crop_no = "Cropped_01"

BASE =  f"/home/neurobiology/drobo/DeepLearningData/083_KN_Exp313-AT/annotations/{gene}/{crop_no}/"
mito_path = f"{BASE}/selected_mito_0-73/"
#mercs_path = f"{BASE}/modified_mercs"
out_dir = f"/home/neurobiology/drobo/DeepLearningData/083_KN_Exp313-AT/Data/{gene}/{crop_no}/"
out_filename = "mito_001_0-73_non3D.csv"

if __name__ == "__main__":
    main(mito_path, out_dir, out_filename)

(74, 1776, 544, 1)


  0%|          | 0/18 [00:00<?, ?it/s]

/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Seri

In [12]:
gene = "01_Control"
crop_no = "Cropped_02"

BASE =  f"/home/neurobiology/drobo/DeepLearningData/083_KN_Exp313-AT/annotations/{gene}/{crop_no}/"
mito_path = f"{BASE}/selected_mito_0-73/"
#mercs_path = f"{BASE}/modified_mercs"
out_dir = f"/home/neurobiology/drobo/DeepLearningData/083_KN_Exp313-AT/Data/{gene}/{crop_no}/"
out_filename = "mito_001_0-73_non3D.csv"

if __name__ == "__main__":
    main(mito_path, out_dir, out_filename)

(74, 1050, 1368, 1)


  0%|          | 0/3 [00:00<?, ?it/s]

/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({


In [13]:
gene = "01_Control"
crop_no = "Cropped_03"

BASE =  f"/home/neurobiology/drobo/DeepLearningData/083_KN_Exp313-AT/annotations/{gene}/{crop_no}/"
mito_path = f"{BASE}/selected_mito_0-73/"
#mercs_path = f"{BASE}/modified_mercs"
out_dir = f"/home/neurobiology/drobo/DeepLearningData/083_KN_Exp313-AT/Data/{gene}/{crop_no}/"
out_filename = "mito_001_0-73_non3D.csv"

if __name__ == "__main__":
    main(mito_path, out_dir, out_filename)

(74, 2448, 1518, 1)


  0%|          | 0/20 [00:00<?, ?it/s]

/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Seri

In [14]:
gene = "01_Control"
crop_no = "Cropped_04"

BASE =  f"/home/neurobiology/drobo/DeepLearningData/083_KN_Exp313-AT/annotations/{gene}/{crop_no}/"
mito_path = f"{BASE}/selected_mito_0-73/"
#mercs_path = f"{BASE}/modified_mercs"
out_dir = f"/home/neurobiology/drobo/DeepLearningData/083_KN_Exp313-AT/Data/{gene}/{crop_no}/"
out_filename = "mito_001_0-73_non3D.csv"

if __name__ == "__main__":
    main(mito_path, out_dir, out_filename)

(74, 1086, 1866, 1)


  0%|          | 0/8 [00:00<?, ?it/s]

/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Seri

In [15]:
gene = "01_Control"
crop_no = "Cropped_05"

BASE =  f"/home/neurobiology/drobo/DeepLearningData/083_KN_Exp313-AT/annotations/{gene}/{crop_no}/"
mito_path = f"{BASE}/selected_mito_0-73/"
#mercs_path = f"{BASE}/modified_mercs"
out_dir = f"/home/neurobiology/drobo/DeepLearningData/083_KN_Exp313-AT/Data/{gene}/{crop_no}/"
out_filename = "mito_001_0-73_non3D.csv"

if __name__ == "__main__":
    main(mito_path, out_dir, out_filename)

(74, 822, 1698, 1)


  0%|          | 0/18 [00:00<?, ?it/s]

/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Seri

In [16]:
gene = "04_PDZD8KO_FKBP8KD"
crop_no = "Cropped_01"

BASE =  f"/home/neurobiology/drobo/DeepLearningData/083_KN_Exp313-AT/annotations/{gene}/{crop_no}/"
mito_path = f"{BASE}/selected_mito/"
#mercs_path = f"{BASE}/modified_mercs"
out_dir = f"/home/neurobiology/drobo/DeepLearningData/083_KN_Exp313-AT/Data/{gene}/{crop_no}/"
out_filename = "mito_001_0-73_non3D.csv"

if __name__ == "__main__":
    main(mito_path, out_dir, out_filename)

(74, 1458, 1710, 1)


  0%|          | 0/41 [00:00<?, ?it/s]

/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Seri

In [17]:
gene = "04_PDZD8KO_FKBP8KD"
crop_no = "Cropped_02"

BASE =  f"/home/neurobiology/drobo/DeepLearningData/083_KN_Exp313-AT/annotations/{gene}/{crop_no}/"
mito_path = f"{BASE}/selected_mito_0-73/"
#mercs_path = f"{BASE}/modified_mercs"
out_dir = f"/home/neurobiology/drobo/DeepLearningData/083_KN_Exp313-AT/Data/{gene}/{crop_no}/"
out_filename = "mito_001_0-73_non3D.csv"

if __name__ == "__main__":
    main(mito_path, out_dir, out_filename)

(115, 702, 2082, 1)


  0%|          | 0/6 [00:00<?, ?it/s]

/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Seri

In [18]:
gene = "04_PDZD8KO_FKBP8KD"
crop_no = "Cropped_03"

BASE =  f"/home/neurobiology/drobo/DeepLearningData/083_KN_Exp313-AT/annotations/{gene}/{crop_no}/"
mito_path = f"{BASE}/selected_mito_0-73/"
#mercs_path = f"{BASE}/modified_mercs"
out_dir = f"/home/neurobiology/drobo/DeepLearningData/083_KN_Exp313-AT/Data/{gene}/{crop_no}/"
out_filename = "mito_001_0-73_non3D.csv"

if __name__ == "__main__":
    main(mito_path, out_dir, out_filename)

(74, 1248, 1888, 1)


  0%|          | 0/2 [00:00<?, ?it/s]

/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({


In [19]:
gene = "04_PDZD8KO_FKBP8KD"
crop_no = "Cropped_04"

BASE =  f"/home/neurobiology/drobo/DeepLearningData/083_KN_Exp313-AT/annotations/{gene}/{crop_no}/"
mito_path = f"{BASE}/selected_mito_0-73/"
#mercs_path = f"{BASE}/modified_mercs"
out_dir = f"/home/neurobiology/drobo/DeepLearningData/083_KN_Exp313-AT/Data/{gene}/{crop_no}/"
out_filename = "mito_001_0-73.csv"

if __name__ == "__main__":
    main(mito_path, out_dir, out_filename)

(74, 1566, 1266, 1)


  0%|          | 0/5 [00:00<?, ?it/s]

/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Seri

In [20]:
gene = "03_FKBP8KD"
crop_no = "Cropped_02"

BASE =  f"/home/neurobiology/drobo/DeepLearningData/083_KN_Exp313-AT/annotations/{gene}/{crop_no}/"
mito_path = f"{BASE}/selected_mito/"
#mercs_path = f"{BASE}/modified_mercs"
out_dir = f"/home/neurobiology/drobo/DeepLearningData/083_KN_Exp313-AT/Data/{gene}/{crop_no}/"
out_filename = "mito_001_0-73_non3D.csv"

if __name__ == "__main__":
    main(mito_path, out_dir, out_filename)

(79, 750, 2520, 1)


  0%|          | 0/8 [00:00<?, ?it/s]

/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Seri

In [21]:
gene = "03_FKBP8KD"
crop_no = "Cropped_03"

BASE =  f"/home/neurobiology/drobo/DeepLearningData/083_KN_Exp313-AT/annotations/{gene}/{crop_no}/"
mito_path = f"{BASE}/selected_mito_0-73/"
#mercs_path = f"{BASE}/modified_mercs"
out_dir = f"/home/neurobiology/drobo/DeepLearningData/083_KN_Exp313-AT/Data/{gene}/{crop_no}/"
out_filename = "mito_001_0-73_non3D.csv"

if __name__ == "__main__":
    main(mito_path, out_dir, out_filename)

(74, 558, 2406, 1)


  0%|          | 0/15 [00:00<?, ?it/s]

/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Seri

In [22]:
gene = "03_FKBP8KD"
crop_no = "Cropped_04"

BASE =  f"/home/neurobiology/drobo/DeepLearningData/083_KN_Exp313-AT/annotations/{gene}/{crop_no}/"
mito_path = f"{BASE}/selected_mito_0-73/"
#mercs_path = f"{BASE}/modified_mercs"
out_dir = f"/home/neurobiology/drobo/DeepLearningData/083_KN_Exp313-AT/Data/{gene}/{crop_no}/"
out_filename = "mito_001_0-73_non3D.csv"

if __name__ == "__main__":
    main(mito_path, out_dir, out_filename)

(79, 672, 4038, 1)


  0%|          | 0/11 [00:00<?, ?it/s]

/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Seri

In [23]:
gene = "03_FKBP8KD"
crop_no = "Cropped_05"

BASE =  f"/home/neurobiology/drobo/DeepLearningData/083_KN_Exp313-AT/annotations/{gene}/{crop_no}/"
mito_path = f"{BASE}/selected_mito_0-73/"
#mercs_path = f"{BASE}/modified_mercs"
out_dir = f"/home/neurobiology/drobo/DeepLearningData/083_KN_Exp313-AT/Data/{gene}/{crop_no}/"
out_filename = "mito_001_0-73_non3D.csv"

if __name__ == "__main__":
    main(mito_path, out_dir, out_filename)

(74, 1428, 3654, 1)


  0%|          | 0/7 [00:00<?, ?it/s]

/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series({
/tmp/ipykernel_18511/1893919530.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Seri

In [ ]:
gene = "03_FKBP8KD"
crop_no = "Cropped_05"

BASE =  f"/home/neurobiology/drobo/DeepLearningData/083_KN_Exp313-AT/annotations/{gene}/{crop_no}/"
mito_path = f"{BASE}/selected_mito/"
#mercs_path = f"{BASE}/modified_mercs"
out_dir = f"/home/neurobiology/drobo/DeepLearningData/083_KN_Exp313-AT/Data/{gene}/{crop_no}/"
out_filename = "mito_001_0-73_non3D.csv"

if __name__ == "__main__":
    main(mito_path, out_dir, out_filename)